In [ ]:
save_fig = True
export_folder = '../output/graph'

# Load benchmark results

In [ ]:
fps_ls = (10, 60, 120)
approach_ls = ('Ours', 'ECPD', 'CPD', 'BCPD')
app2code = {
    'Ours': 'rbf',
    'ECPD': 'ecpd',
    'CPD': 'cpd',
    'BCPD': 'bcpd',
}

In [ ]:
import os
from mesh4d import utils
from benchmark import Bemchmarker_marker_guided, Bemchmarker_marker_less

benchmarker_dict = {}

for fps in fps_ls:
    benchmarker_dict[fps] = {}

    for approach in approach_ls:
        path = os.path.join('../output/', f'{fps}fps', app2code[approach], 'benchmark.pkl')
        benchmarker = utils.load_pkl_object(path)
        benchmarker_dict[fps][approach] = benchmarker
    
benchmarker_dict

# Tables

In [ ]:
def get_duration(benchmarker = None):
    if benchmarker is None:
        return 'time (s)'

    else:
        return benchmarker.duration

In [ ]:
import numpy as np

def get_acc_control(benchmarker = None):
    if benchmarker is None:
        return 'acc-c (cm)'

    else:
        if type(benchmarker) == Bemchmarker_marker_less:
            return np.nan
        
        else:
            return benchmarker.diff_control['dist_mean'] / 10

In [ ]:
def get_acc_noncontrol(benchmarker = None):
    if benchmarker is None:
        return 'acc-nc (cm)'

    else:
        if type(benchmarker) == Bemchmarker_marker_less:
            return benchmarker.diff_noncontrol['dist_mean']
        
        else:
            dist_ls = [diff['dist_mean'] for diff in benchmarker.diff_noncontrol.values()]
            return np.mean(dist_ls) / 10

In [ ]:
import pandas as pd

# init data frame
df = pd.DataFrame(columns=('Dataset', 'Metric') + approach_ls)

# fill in data
row = 0
get_metric_ls = [get_duration, get_acc_control, get_acc_noncontrol]

for fps in fps_ls:
    for get_metric in get_metric_ls:
        row_data = [f'DBL-{fps}', get_metric()]

        for approach in approach_ls:
            matric = get_metric(benchmarker_dict[fps][approach])
            row_data.append(matric)
        
        df.loc[row] = row_data
        row = row + 1

df

In [ ]:
df_styler = df.set_index(['Dataset', 'Metric']).style
df_styler.format(
    precision=2, 
    na_rep='--',
    )
df_styler.highlight_min(
    axis=1, 
    props='bfseries: ;'
    )
df_styler

In [ ]:
print(df_styler.to_latex(
    position='h',
    position_float='centering',
    hrules=True,
    # label='',
    # caption='',
))

# Plots

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 600

app2col = {
    'Ours': 'goldenrod',
    'ECPD': 'teal',
    'CPD': 'olive',
    'BCPD': 'hotpink',
    }

## Computation time

In [ ]:
df_time = df[df['Metric']=='time (s)'].set_index('Dataset')
df_time

In [ ]:
for approach in df_time.columns[1:]:
    plt.plot(df_time[approach], label=approach, marker='.', color=app2col[approach])

plt.ylabel('Computation Time (s)')
plt.legend()

if save_fig:
    plt.savefig(os.path.join(export_folder, f'compute_time.png'))

plt.show()

## Alignment on control landmarks

In [ ]:
def get_dist_control(benchmarker):
    dist_ls = []

    for marker in benchmarker.diff_control['diff_dict'].values():
        dist_ls.append(marker['dist'])

    return np.array(dist_ls)/10

In [ ]:
data = []
labels = []
colors = []

approach_ls_here = ('Ours', 'ECPD')

for approach in approach_ls_here:
    for fps in fps_ls:
        labels.append(f'DBL-{fps}')
        colors.append(app2col[approach])
        data.append(get_dist_control(benchmarker_dict[fps][approach]).reshape((-1,)))

bplot = plt.boxplot(
    x = data, labels = labels,
    patch_artist=True, showmeans=True, meanline=True, sym='',
    medianprops=dict(color='black'), meanprops=dict(color='black'),
    )

for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)

plt.ylabel('Alignment Error (cm)')
plt.legend(
    bplot['boxes'][0::len(fps_ls)], 
    approach_ls_here
    )

if save_fig:
    plt.savefig(os.path.join(export_folder, f'align_control.png'))

plt.show()

In [ ]:
num = 0

for fps in fps_ls:
    num = num + 1
    ax = plt.subplot(310 + num)

    for approach in approach_ls_here:
        dist = get_dist_control(benchmarker_dict[fps][approach]).mean(axis=0)
        plt.plot(
            np.linspace(0, 1, len(dist)), dist,
            label=approach, color=app2col[approach],
            )
    
    plt.legend(fontsize=7)

    if num == round(len(fps_ls)/2):
        plt.ylabel('Alignment Error (cm)')

    if num < len(fps_ls):
        plt.tick_params('x', labelbottom=False)
    else:
        plt.xlabel('Time (s)')

if save_fig:
    plt.savefig(os.path.join(export_folder, f'align_control_time.png'))

plt.show()

## Alignment on non-control landmarks

In [ ]:
def get_dist_noncontrol(benchmarker):
    dist_ls = []

    if type(benchmarker) == Bemchmarker_marker_less:
        for marker in benchmarker.diff_noncontrol['diff_dict'].values():
            dist_ls.append(marker['dist'])

    elif type(benchmarker) == Bemchmarker_marker_guided:
        for name, marker in benchmarker.diff_noncontrol.items():   
            dist_ls.append(marker['diff_dict'][name]['dist'])

    return np.array(dist_ls)/10

In [ ]:
data = []
labels = []
colors = []

for approach in approach_ls:
    for fps in fps_ls:
        labels.append(f'DBL-{fps}')
        colors.append(app2col[approach])
        data.append(get_dist_noncontrol(benchmarker_dict[fps][approach]).reshape((-1,)))

bplot = plt.boxplot(
    x = data, labels = labels,
    patch_artist=True, showmeans=True, meanline=True, sym='',
    medianprops=dict(color='black'), meanprops=dict(color='black'),
    )

for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)

plt.gca().set_aspect(1/3)
plt.xticks(fontsize=7)
plt.ylabel('Alignment Error (cm)')
plt.legend(
    bplot['boxes'][0::len(fps_ls)], 
    approach_ls
    )

if save_fig:
    plt.savefig(os.path.join(export_folder, f'align_noncontrol.png'))

plt.show()

In [ ]:
num = 0

for fps in fps_ls:
    num = num + 1
    ax = plt.subplot(310 + num)

    for approach in approach_ls:
        dist = get_dist_noncontrol(benchmarker_dict[fps][approach]).mean(axis=0)
        plt.plot(
            np.linspace(0, 1, len(dist)), dist,
            label=approach, color=app2col[approach],
            )
    
    plt.legend(fontsize=7)
    plt.title(f'DBL-{fps}', fontsize=7)

    if num == round(len(fps_ls)/2):
        plt.ylabel('Alignment Error (cm)')

    if num < len(fps_ls):
        plt.tick_params('x', labelbottom=False)
    else:
        plt.xlabel('Time (s)')

if save_fig:
    plt.savefig(os.path.join(export_folder, f'align_noncontrol_time.png'))

plt.show()